In [0]:
df_hosa = spark.read.parquet("/mnt/bronze/hosa/transactions")
df_hosa.createOrReplaceTempView("hosa_transactions")

df_hosb = spark.read.parquet("/mnt/bronze/hosb/transactions")
df_hosb.createOrReplaceTempView("hosb_transactions")

In [0]:
%sql
select  * from hosa_transactions

In [0]:
%sql
select * from hosb_transactions

In [0]:
for col1 in df_hosa.columns:
  for col2 in df_hosb.columns:
    if col1 == col2:
      print(col1)
    else:
        pass

In [0]:
%sql
create or replace temp view cdm_transactions as
select concat(TransactionID, '-',datasource)as SRC_Transaction_Key, * from
 (
select * from hosa_transactions
union all
select * from hosb_transactions
)

In [0]:
%sql
select * from cdm_transactions

In [0]:
%sql
create or replace temp view quality_checks as (
    SELECT 
     concat(TransactionID,'-',datasource) as TransactionID,
     TransactionID as SRC_TransactionID,
     EncounterID,
     PatientID,
     ProviderID,
     DeptID,
     VisitDate,
     ServiceDate,
     PaidDate,
     VisitType,
     Amount,
     AmountType,
     PaidAmount,
     ClaimID,
     PayorID,
     ProcedureCode,
     ICDCode,
     LineOfBusiness,
     MedicaidID,
     MedicareID,
     InsertDate as SRC_InsertDate,
     ModifiedDate as SRC_ModifiedDate,
     datasource,
         CASE 
             WHEN EncounterID IS NULL OR PatientID IS NULL OR TransactionID IS NULL OR VisitDate IS NULL THEN TRUE
             ELSE FALSE
         END AS is_quarantined

from cdm_transactions
)

In [0]:
%sql
select * from quality_checks

In [0]:
 %sql
 CREATE TABLE IF NOT EXISTS silver.transactions (
   TransactionID string,
   SRC_TransactionID string,
   EncounterID string,
   PatientID string,
   ProviderID string,
   DeptID string,
   VisitDate date,
   ServiceDate date,
   PaidDate date,
   VisitType string,
   Amount double,
   AmountType string,
   PaidAmount double,
   ClaimID string,
   PayorID string,
   ProcedureCode integer,
   ICDCode string,
   LineOfBusiness string,
   MedicaidID string,
   MedicareID string,
   SRC_InsertDate date,
   SRC_ModifiedDate date,
   datasource string,
   is_quarantined boolean,
   audit_insertdate timestamp,
   audit_modifieddate timestamp,
   is_current boolean
 )
 USING DELTA;

In [0]:
%sql
merge into silver.transactions as target
using quality_checks as source
on target.TransactionID == source.TransactionID
and target.is_current = true
when matched 
and(
     target.SRC_TransactionID != source.SRC_TransactionID
  or target.EncounterID != source.EncounterID
  or target.PatientID != source.PatientID
  or target.ProviderID != source.ProviderID
  or target.DeptID != source.DeptID
  or target.VisitDate != source.VisitDate
  or target.ServiceDate != source.ServiceDate
  or target.PaidDate != source.PaidDate
  or target.VisitType != source.VisitType
  or target.Amount != source.Amount
  or target.AmountType != source.AmountType
  or target.PaidAmount != source.PaidAmount
  or target.ClaimID != source.ClaimID
  or target.PayorID != source.PayorID
  or target.ProcedureCode != source.ProcedureCode
  or target.ICDCode != source.ICDCode
  or target.LineOfBusiness != source.LineOfBusiness
  or target.MedicaidID != source.MedicaidID
  or target.MedicareID != source.MedicareID
  or target.SRC_InsertDate != source.SRC_InsertDate
  or target.SRC_ModifiedDate != source.SRC_ModifiedDate
  or target.datasource != source.datasource
  or target.is_quarantined != source.is_quarantined
)
then update set target.audit_modifieddate = current_timestamp(),
                target.is_current = false

when not matched

then insert (
   TransactionID,
   SRC_TransactionID,
   EncounterID,
   PatientID,
   ProviderID,
   DeptID,
   VisitDate,
   ServiceDate,
   PaidDate,
   VisitType,
   Amount,
   AmountType,
   PaidAmount,
   ClaimID,
   PayorID,
   ProcedureCode,
   ICDCode,
   LineOfBusiness,
   MedicaidID,
   MedicareID,
   SRC_InsertDate,
   SRC_ModifiedDate,
   datasource,
   is_quarantined,
   audit_insertdate,
   audit_modifieddate,
   is_current
) values (
   source.TransactionID,
   source.SRC_TransactionID,
   source.EncounterID,
   source.PatientID,
   source.ProviderID,
   source.DeptID,
   source.VisitDate,
   source.ServiceDate,
   source.PaidDate,
   source.VisitType,
   source.Amount,
   source.AmountType,
   source.PaidAmount,
   source.ClaimID,
   source.PayorID,
   source.ProcedureCode,
   source.ICDCode,
   source.LineOfBusiness,
   source.MedicaidID,
   source.MedicareID,
   source.SRC_InsertDate,
   source.SRC_ModifiedDate,
   source.datasource,
   source.is_quarantined,
   current_timestamp(),
   current_timestamp(),
   true
)